# Interacting with SQL database through python and pandas



When you're working with small enough datasets, you can simply load the data into memmory and work with it in python. But sometimes you have so much data that loading it into memory is either impossible or very slow. In that case, storing your data in a [SQL](https://en.wikipedia.org/wiki/SQL) database might be a good option. A SQL database allows you to run queries on large datasets much more efficiently than if the data was stored in csv format.

I’m currently working on a project where the amount of data was too much to work with in python all at once, so I decided to instead store my data in a SQL database. I'm doing my analysis and visualization in python, so I also wanted to learn how to do all this from python. In this post, i'll go over an example of how to add data to a SQL database and query the database in python using Pandas. It’s actually very easy!

First you need to create a database to add to or read from. I’m using [SQLite](https://en.wikipedia.org/wiki/SQLite), because it is simple and easy, but you can work with pretty much any SQL variant in python (for working with types other than SQLite in Pandas, you need the SQLAlchemy package (<https://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries>). I created my empty database in [SQLiteStudio](https://sqlitestudio.pl/index.rvt). 

Next we need to add some tables and data. You could create the tables in SQLiteStudio first and then add data to them, but I will create the tables from python. This is nice because it automatically names all the columns, and you have a script that will reproduce everything. 

First we need to open a connection to the database. My databse is called 'sqlite_example.db3'. We import the sqlite3 package and use it to connect to the database.

In [1]:
import sqlite3 # use a different test database??
con = sqlite3.connect("/Users/Andy/sqlite_example.db3")

Next i'll read in some of the data i'm working with, historical ridership for the citibike bike-rental program in NYC. I've already downloaded the data as a csv file.

In [2]:
import pandas as pd
df = pd.read_csv('/Users/Andy/2014-03 - Citi Bike trip data.csv',parse_dates=True)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,949,2014-03-01 00:00:16,2014-03-01 00:16:05,317,E 6 St & Avenue B,40.724537,-73.981854,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,17440,Subscriber,1942,1
1,533,2014-03-01 00:00:57,2014-03-01 00:09:50,457,Broadway & W 58 St,40.766953,-73.981693,441,E 52 St & 2 Ave,40.756014,-73.967416,20855,Subscriber,1960,1
2,122,2014-03-01 00:01:06,2014-03-01 00:03:08,146,Hudson St & Reade St,40.716250,-74.009106,276,Duane St & Greenwich St,40.717488,-74.010455,15822,Subscriber,1984,1
3,134,2014-03-01 00:01:14,2014-03-01 00:03:28,146,Hudson St & Reade St,40.716250,-74.009106,276,Duane St & Greenwich St,40.717488,-74.010455,17793,Subscriber,1985,1
4,997,2014-03-01 00:01:18,2014-03-01 00:17:55,150,E 2 St & Avenue C,40.720874,-73.980858,461,E 20 St & 2 Ave,40.735877,-73.982050,20756,Subscriber,1977,1


Now I want to store this data in my SQL database. We can create a SQL table w/ the Pandas method **to_sql**.

In [3]:
df.to_sql("rides",con,if_exists='replace',index=False) 

/Users/Andy/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1201: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  chunksize=chunksize, dtype=dtype)


This will write the data frame to a table called “rides”. The *if_exists* parameter specifies what to do if a table of the same name already exists. 
- '*replace*' will replace the pre-existing table. 
- '*fail*' will return an error (useful if you want to be sure you don’t overwrite a table). - - '*append*' will add to the table.

Now we need to read data from the database. This is done w/ the pandas **read_sql_query** method. For example, if I want to get all the rows where tripduration was less than 500, I would use:

In [4]:
df = pd.read_sql_query("SELECT * FROM rides WHERE tripduration < 500 ",con)
df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,122,2014-03-01 00:01:06,2014-03-01 00:03:08,146,Hudson St & Reade St,40.716250,-74.009106,276,Duane St & Greenwich St,40.717488,-74.010455,15822,Subscriber,1984,1
1,134,2014-03-01 00:01:14,2014-03-01 00:03:28,146,Hudson St & Reade St,40.716250,-74.009106,276,Duane St & Greenwich St,40.717488,-74.010455,17793,Subscriber,1985,1
2,231,2014-03-01 00:02:08,2014-03-01 00:05:59,384,Fulton St & Waverly Ave,40.683178,-73.965964,399,Lafayette Ave & St James Pl,40.688515,-73.964763,20117,Subscriber,1982,1
3,387,2014-03-01 00:02:24,2014-03-01 00:08:51,521,8 Ave & W 31 St,40.750450,-73.994811,529,W 42 St & 8 Ave,40.757570,-73.990985,18856,Subscriber,1975,2
4,115,2014-03-01 00:02:28,2014-03-01 00:04:23,438,St Marks Pl & 1 Ave,40.727791,-73.985649,438,St Marks Pl & 1 Ave,40.727791,-73.985649,20922,Subscriber,1994,2


Note you don’t actually have to capitalize the SQL query commands, but it is standard practice, and makes them much easier to read. The nice thing about using this method to query the databse is that it returns the results of the query in a Pandas dataframe, which you can then easily manipulate or analyze.

In this example i’ve only used one month of data, which could could have handled in memory. But in the project i’m working on, i’ll be analyzing years of data, which is much too large to load into memory. One option would be to read it in chunks and perform operations on each chunk. But that is clunky and would require re-reading the data for every new query I want to make. Storing the data in a SQL database allows me to easily and quickly query the entire dataset, and have the results immediately available for analysis in python. 



Some resources I found helpful:
- https://www.dataquest.io/blog/python-pandas-databases/
- http://www.datacarpentry.org/python-ecology-lesson/08-working-with-sql/
- https://pandas.pydata.org/pandas-docs/stable/io.html#sql-queries